# Loading Libraries

Connect to Earth engine, import packages.

In [ ]:
 import ee
!pip install geopandas geemap geehydro
import geemap
import folium
import os
import geehydro

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize(project='ccri-chla-modeling')

# from google.colab import drive
# drive.mount('/content/drive')
# out_dir = os.path.expanduser('/content/drive/test')


Import assets from gee

In [ ]:
## shapefile (STORETID) table as csv
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io

#assets from GEE
UniquePts = ee.FeatureCollection("users/greeneji/MiCorp_Secchi_4ha_dissolve")

STORETIDs = pd.read_csv(io.BytesIO(uploaded['MiCorp_Secchi_4ha_dissolve.csv']))
STORETIDs1 = STORETIDs.iloc[0:, 1].dropna().to_list()



# MAIN Atmospheric Correction

Page, B.P., Olmanson, L.G. and Mishra, D.R., 2019. A harmonized image processing workflow using Sentinel-2/MSI and Landsat-8/OLI for mapping water clarity in optically variable lake systems. Remote Sensing of Environment, 231, p.111284.

https://github.com/Nateme16/geo-aquawatch-water-quality/blob/main/Atmospheric%20corrections/main_L8L9.ipynb

In [ ]:
# MAIN Atmospheric Correction

def atm_corr(img):

    footprint = img.geometry()

    # DEM
    dem = ee.Image('USGS/SRTMGL1_003').clip(footprint)
    DEM_OLI = ee.Image(1);


    # ozone
    # DU_OLI = ee.Image(ozone.filterBounds(footprint).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month')).filter(ee.Filter.calendarRange(startYear, endYear, 'year')).mean())

    DU_OLI = ee.Image(300); # ozone @ sea level

    # Julian Day
    imgDate_OLI = ee.Date(img.get('system:time_start'))
    FOY_OLI = ee.Date.fromYMD(imgDate_OLI.get('year'),1,1)
    JD_OLI = imgDate_OLI.difference(FOY_OLI,'day').int().add(1)

    # Earth-Sun distance
    d_OLI = ee.Image.constant(img.get('EARTH_SUN_DISTANCE'))

    # Sun elevation
    SunEl_OLI = ee.Image.constant(img.get('SUN_ELEVATION'))

    # Sun azimuth
    SunAz_OLI = img.select('SAA').multiply(ee.Image(0.01))

    # Satellite zenith
    SatZe_OLI = img.select('VZA').multiply(ee.Image(0.01))
    cosdSatZe_OLI = (SatZe_OLI).multiply(pi.divide(ee.Image(180))).cos()
    sindSatZe_OLI = (SatZe_OLI).multiply(pi.divide(ee.Image(180))).sin()

    # Satellite azimuth
    SatAz_OLI = img.select('VAA').multiply(ee.Image(0.01))

    # Sun zenith
    SunZe_OLI = img.select('SZA').multiply(ee.Image(0.01))
    cosdSunZe_OLI = SunZe_OLI.multiply(pi.divide(ee.Image.constant(180))).cos() # in degrees
    sindSunZe_OLI = SunZe_OLI.multiply(pi.divide(ee.Image(180))).sin() # in degrees

    # Relative azimuth
    RelAz_OLI = ee.Image(SunAz_OLI)
    cosdRelAz_OLI = RelAz_OLI.multiply(pi.divide(ee.Image(180))).cos()

    # Pressure calculation
    P_OLI = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM_OLI)).pow(5.25588)).multiply(0.01)
    Po_OLI = ee.Image(1013.25)

    # Radiometric Calibration
    # define bands to be converted to radiance
    bands_OLI = ['B1','B2','B3','B4','B5','B6','B7']

    # radiance_mult_bands
    rad_mult_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_MULT_BAND_1')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_2')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_3')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_4')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_5')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_6')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_7'))]
                        )).toArray(1)

    # radiance add band
    rad_add_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_ADD_BAND_1')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_2')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_3')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_4')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_5')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_6')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_7'))]
                        )).toArray(1)

    # create an empty image to save new radiance bands to
    imgArr_OLI = img.select(bands_OLI).toArray().toArray(1);
    Ltoa_OLI = imgArr_OLI.multiply(rad_mult_OLI).add(rad_add_OLI)

    # print(Ltoa_OLI)

    # esun (extra-terrestrial solar irradiance) Units = mW cm-2 um-1
    ESUN_OLI = ee.Image.constant(197.24790954589844).\
      addBands(ee.Image.constant(201.98426818847656)).\
      addBands(ee.Image.constant(186.12677001953125)).\
      addBands(ee.Image.constant(156.95257568359375)).\
      addBands(ee.Image.constant(96.04714965820312)).\
      addBands(ee.Image.constant(23.8833221450863)).\
      addBands(ee.Image.constant(8.04995873449635)).toArray().toArray(1)
    ESUN_OLI = ESUN_OLI.multiply(ee.Image(1))

    ESUNImg_OLI = ESUN_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Ozone Correction
    # Ozone coefficients https://www.arm.gov/publications/tech_reports/doe-sc-arm-tr-129.pdf?id=811 (Appendix A) by band center (lambda)
    koz_OLI = ee.Image.constant(0.0039).addBands(ee.Image.constant(0.0218)).\
      addBands(ee.Image.constant(0.1078)).addBands(ee.Image.constant(0.0608)).addBands(ee.Image.constant(0.0019)).addBands(ee.Image.constant(0)).addBands(ee.Image.constant(0)).toArray().toArray(1)

    # Calculate ozone optical thickness
    Toz_OLI = koz_OLI.multiply(DU_OLI).divide(ee.Image.constant(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt_OLI = Ltoa_OLI.multiply(((Toz_OLI)).multiply((ee.Image.constant(1).divide(cosdSunZe_OLI)).add(ee.Image.constant(1).divide(cosdSatZe_OLI))).exp())

    # Rayleigh optical thickness
    bandCenter_OLI = ee.Image(443).divide(1000).addBands(ee.Image(483).divide(1000)).addBands(ee.Image(561).divide(1000)).addBands(ee.Image(655).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1609).divide(1000)).addBands(ee.Number(2201).divide(1000)).toArray().toArray(1)

     # create an empty image to save new Tr values to
    Tr_OLI = (P_OLI.divide(Po_OLI)).multiply(ee.Image(0.008569).multiply(bandCenter_OLI.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter_OLI.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter_OLI.pow(-4)))))

    # Fresnel Reflection
    # Specular reflection (s- and p- polarization states)
    theta_V_OLI = ee.Image(0.0000000001)
    sin_theta_j_OLI = sindSunZe_OLI.divide(ee.Image(1.333))

    theta_j_OLI = sin_theta_j_OLI.asin().multiply(ee.Image(180).divide(pi))

    theta_SZ_OLI = SunZe_OLI

    R_theta_SZ_s_OLI = (((theta_SZ_OLI.multiply(pi.divide(ee.Image(180)))).subtract(theta_j_OLI.multiply(pi.divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ_OLI.multiply(pi.divide(ee.Image(180)))).add(theta_j_OLI.multiply(pi.divide(ee.Image(180))))).sin().pow(2)))

    R_theta_V_s_OLI = ee.Image(0.0000000001)

    R_theta_SZ_p_OLI = (((theta_SZ_OLI.multiply(pi.divide(180))).subtract(theta_j_OLI.multiply(pi.divide(180)))).tan().pow(2)).divide((((theta_SZ_OLI.multiply(pi.divide(180))).add(theta_j_OLI.multiply(pi.divide(180)))).tan().pow(2)))

    R_theta_V_p_OLI = ee.Image(0.0000000001)

    R_theta_SZ_OLI = ee.Image(0.5).multiply(R_theta_SZ_s_OLI.add(R_theta_SZ_p_OLI))

    R_theta_V_OLI = ee.Image(0.5).multiply(R_theta_V_s_OLI.add(R_theta_V_p_OLI))

    # Rayleigh scattering phase function
    # Sun-sensor geometry

    theta_neg_OLI = ((cosdSunZe_OLI.multiply(ee.Image(-1))).multiply(cosdSatZe_OLI)).subtract((sindSunZe_OLI).multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_neg_inv_OLI = theta_neg_OLI.acos().multiply(ee.Image(180).divide(pi))

    theta_pos_OLI = (cosdSunZe_OLI.multiply(cosdSatZe_OLI)).subtract(sindSunZe_OLI.multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_pos_inv_OLI = theta_pos_OLI.acos().multiply(ee.Image(180).divide(pi))

    cosd_tni_OLI = theta_neg_inv_OLI.multiply(pi.divide(180)).cos() # in degrees

    cosd_tpi_OLI = theta_pos_inv_OLI.multiply(pi.divide(180)).cos() # in degrees

    Pr_neg_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni_OLI.pow(2))))

    Pr_pos_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi_OLI.pow(2))))

    # Rayleigh scattering phase function
    Pr_OLI = Pr_neg_OLI.add((R_theta_SZ_OLI.add(R_theta_V_OLI)).multiply(Pr_pos_OLI));

    # Calulate Lr,
    denom_OLI = ee.Image(4).multiply(pi).multiply(cosdSatZe_OLI)
    Lr_OLI = (ESUN_OLI.multiply(Tr_OLI)).multiply(Pr_OLI.divide(denom_OLI))

    # Rayleigh corrected radiance
    Lrc_OLI = (Lt_OLI.divide(ee.Image(10))).subtract(Lr_OLI)
    LrcImg_OLI = Lrc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rayleigh corrected reflectance
    prc_OLI = Lrc_OLI.multiply(pi).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI))
    prcImg_OLI = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])
    rhorc = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Aerosol Correction
    # Bands in nm
    bands_nm_OLI = ee.Image(443).addBands(ee.Image(483)).addBands(ee.Image(561)).addBands(ee.Image(655)).addBands(ee.Image(865)).addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1)

    # Lam in SWIR bands
    Lam_6_OLI = LrcImg_OLI.select('B6')
    Lam_7_OLI = LrcImg_OLI.select('B7')

    # Calculate aerosol type
    eps_OLI = (((((Lam_7_OLI).divide(ESUNImg_OLI.select('B7'))).log()).subtract(((Lam_6_OLI).divide(ESUNImg_OLI.select('B6'))).log())).divide(ee.Image(2201).subtract(ee.Image(1609)))) #.multiply(water_mask)

    # Calculate multiple scattering of aerosols for each band
    Lam_OLI = (Lam_7_OLI).multiply(((ESUN_OLI).divide(ESUNImg_OLI.select('B7')))).multiply((eps_OLI.multiply(ee.Image(-1))).multiply((bands_nm_OLI.divide(ee.Image(2201)))).exp())

    # diffuse transmittance
    trans_OLI = Tr_OLI.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe_OLI)).exp()

    # Compute water-leaving radiance
    Lw_OLI = Lrc_OLI.subtract(Lam_OLI).divide(trans_OLI)

    # water-leaving reflectance
    pw_OLI = (Lw_OLI.multiply(pi).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI)))
    pwImg_OLI = pw_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rrs
    Rrs = (pw_OLI.divide(pi).arrayProject([0]).arrayFlatten([bands_OLI]).slice(0,5)).multiply(mask)
    Rrs = Rrs.updateMask(Rrs.gt(0));

    return Rrs.set('system:time_start',img.get('system:time_start'))


# Secchi Code

## Creating mask functions

These functions mask clouds based on the QA_PIXEL band (maskL8sr), select pixels that are >= 75% water (jrcMask), and a 30m buffer around roads to mask bridges (roadMask)

In [ ]:
#Creating mask functions
# function to mask out clouds
def maskL8sr(image):
  # Bits 2, 3, and 4, are cirrus, cloud  and cloudshadow, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 4)
  cirrusBitMask = (1 << 2)
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)

# jrc water occurrence mask
def jrcMask(image):
  jrc = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
  # select only water occurence
  occurrence = jrc.select('occurrence')
  # selectonly water occurences of greater than 75%
  water_mask = occurrence.mask(occurrence.gt(50))
  return image.updateMask(water_mask)

#Creating 30m road buffer mask
def roadMask(image):
  roads = ee.FeatureCollection("TIGER/2016/Roads")
  # 30m road buffer
  def bufferPoly30(feature):
    return feature.buffer(30)
  Buffer = roads.map(bufferPoly30)
    # Convert 'areasqkm' property from string to number.
  def func_uem(feature):
        num = ee.Number.parse(ee.String(feature.get('linearid')))
        return feature.set('linearid', num)
  roadBuffer = Buffer.map(func_uem)
  roadRaster = roadBuffer.reduceToImage(['linearid'], ee.Reducer.first())
  # create an image with a constant value of one to apply roadmask to
  blank = ee.Image.constant(1)
  inverseMask = blank.updateMask(roadRaster)
  # get reverse mask to have everything but roads kept
  mask = inverseMask.mask().Not()
  return image.updateMask(mask)

Create filters to select summer dates

In [ ]:
sum13 = ee.Filter.date('2013-05-01','2013-09-30')
sum14 = ee.Filter.date('2014-05-01','2014-09-30')
sum15 = ee.Filter.date('2015-05-01','2015-09-30')
sum16 = ee.Filter.date('2016-05-01','2016-09-30')
sum17 = ee.Filter.date('2017-05-01','2017-09-30')
sum18 = ee.Filter.date('2018-05-01','2018-09-30')
sum19 = ee.Filter.date('2019-05-01','2019-09-30')
sum20 = ee.Filter.date('2020-05-01','2020-09-30')
sum21 = ee.Filter.date('2021-05-01','2021-09-30')
sum22 = ee.Filter.date('2022-05-01','2022-09-30')
sum23 = ee.Filter.date('2023-05-01','2023-09-30')
Summers = ee.Filter.Or(sum13, sum14, sum15, sum16, sum17, sum18, sum19, sum20, sum21, sum22, sum23)

In [ ]:
#create point buffer for every pt
def wrap_buffer(pt_collection, radius):
  def bufferPoints(pt):
    pt_buffer = pt.buffer(radius).bounds()
    return pt_buffer
  return pt_collection.map(lambda ptfeat: bufferPoints(ptfeat))

ptBuffer = wrap_buffer(UniquePts, 60)

In [ ]:
#atm correction parameters
geometry = ptBuffer
JRC = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
mask = JRC.select('occurrence').gt(50)

# oliCloudPerc = 50

target_image_number = 1

## Import collections


In [ ]:
# Import Collections

ozone = ee.ImageCollection('TOMS/MERGED')
pi = ee.Image(3.141592);

In [ ]:
# filter landsat 8 and 9 scenes by path / row
FC_OLI = ee.ImageCollection("LANDSAT/LC08/C02/T1") \
                .filter(Summers) \
                .filterBounds(UniquePts) \
                .filterMetadata('CLOUD_COVER', "less_than", 50) \
                .map(maskL8sr) \
                .map(jrcMask) \
                .map(roadMask) \
                .sort('system:time_start')

FC_OLI2 = ee.ImageCollection("LANDSAT/LC09/C02/T1") \
                .filter(Summers) \
                .filterBounds(UniquePts) \
                .filterMetadata('CLOUD_COVER', "less_than", 50) \
                .map(maskL8sr) \
                .map(jrcMask) \
                .map(roadMask) \
                .sort('system:time_start')

FC_combined = FC_OLI.merge(FC_OLI2).sort('system:time_start')

#print(FC_combined, 'Available Imagery')
# fcList = FC_combined.toList(10000)
# print(FC_OLI.size().getInfo())
# print(FC_combined.size().getInfo())

In [ ]:
# Processing Collection
Rrs = FC_combined.map(atm_corr).sort('system:time_start')
#print(Rrs, 'Rrs')
# Rrs_list = Rrs.toList(100000)

RrsSelect = Rrs \
  .select(['B1', 'B2', 'B3', 'B4', 'B5'])


### Zonal Statistics by STORET ID

Creates a file for each STORET_ID.


convert Rrs to 32-bit integer

In [ ]:
from geemap.geemap import ee_to_pandas
import gc
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

STORETIDs1 = STORETIDs.iloc[0:, 1].dropna().to_list()

years = list(range(2013, 2024, 1))

df1 = pd.DataFrame()

for station in STORETIDs1:
  #for yr in years:
    print(station)
    filteredBuffer = ee.FeatureCollection(ptBuffer.filter(ee.Filter.eq('STORETID', station)))
    datasetBands_filt = RrsSelect.filterBounds(filteredBuffer)

    out_landsat_stats = os.path.join(out_dir, 'MAIN_L8_lakes_' + str(station) +'.csv')
    zonalstats_out = geemap.zonal_statistics(datasetBands_filt, filteredBuffer , out_landsat_stats, statistics_type='MEDIAN', scale=30, return_fc = True)
    df2 = geemap.ee_to_geopandas(zonalstats_out)
    df1 = pd.concat([df1,df2])

    print("Complete")

output_filepath = os.path.join(out_dir, 'MAIN_L8_secchilakes_stats.csv')
df1.to_csv(output_filepath)